1) where the hotspots are currently and what areas to focus on next on a map.

2) How would you calculate the fatality rate and what issues do you see with presenting those figures based on this dataset?

3) Please cite any additional sources you used for research and reference any existing dashboards you used for inspiration
 

In [175]:
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta

# load data 

rename some columns name, drop some un-necessary columns in order to merge the tables later 

In [176]:
src_path = {'wc':'time_series_covid19_confirmed_global.csv',\
            'wd':'time_series_covid19_deaths_global.csv',\
            'uc':'time_series_covid19_confirmed_US.csv',\
            'ud':'time_series_covid19_deaths_US.csv',\
            'geo':'UID_ISO_FIPS_LookUp_Table.csv'}

target_path = {'summary':'time_series_covid19.csv'}

In [177]:
# Read global data
wc = pd.read_csv("csse_covid_19_time_series/"+src_path['wc'])
wd = pd.read_csv("csse_covid_19_time_series/"+src_path['wd'])

# Geographical information will be merged
wc.drop(['Lat','Long'],axis=1,inplace=True)
wd.drop(['Lat','Long'],axis=1,inplace=True)

# Read US data
uc = pd.read_csv("csse_covid_19_time_series/"+src_path['uc'])
ud = pd.read_csv("csse_covid_19_time_series/"+src_path['ud'])

# rename some columns name so that all the tables have the same names 
uc.rename(columns={'Province_State':'Province/State','Country_Region':'Country/Region','Long_':'Long'},inplace=True)
ud.rename(columns={'Province_State':'Province/State','Country_Region':'Country/Region','Long_':'Long'},inplace=True)

# County-level data to be purged
uc.drop(['UID','iso2','code3','Admin2','FIPS','Combined_Key'],axis=1,inplace=True)
ud.drop(['UID','iso2','code3','Admin2','FIPS','Combined_Key'],axis=1,inplace=True)

# Append Population column to infections data
uc = pd.concat([uc.iloc[:,:5],ud['Population'],uc.iloc[:,5:]],axis=1)

# Read geographical information
geo = pd.read_csv(src_path['geo'])
geo.rename(columns={'Province_State':'Province/State','Country_Region':'Country/Region','Long_':'Long'},inplace=True)
geo.drop(['UID','iso2','code3','Admin2','FIPS','Combined_Key'],axis=1,inplace=True)

# clean data

In [178]:
# order the columns of each table into same order 
def reorder_columns(cols):
    # iso
    # Province/State
    # Country/Region
    # Lat
    # Long
    # Population
    lst = cols
    bases_idx = [cols.get_loc('iso3'),cols.get_loc('Province/State'),cols.get_loc('Country/Region'),cols.get_loc('Lat'),cols.get_loc('Long'),cols.get_loc('Population')]
    bases = cols[bases_idx]

    return list(bases) + list(lst.drop(bases))

In [179]:
cutoff=815  # cutoff = number of days frmo 4/7/22 

# Get least recent date (reporting time gaps)
min_latest = datetime.strptime(min(wc.columns[-1],wd.columns[-1],uc.columns[-1],ud.columns[-1]),'%m/%d/%y')
max_first = datetime.strptime(max(wc.columns[2],wd.columns[2],uc.columns[6],ud.columns[6]),'%m/%d/%y')
min_date = max(min_latest-timedelta(days=cutoff),max_first)

start_date = min_date.strftime('%-m/%-d/%y')
end_date = min_latest.strftime('%-m/%-d/%y')

In [180]:
print(start_date)
print(end_date)

1/22/20
4/7/22


In [181]:
col_list = {'wc':list(wc.columns[:2]) \
            +list(wc.columns[wc.columns.get_loc(start_date):wc.columns.get_loc(end_date)+1]),
            'wd':list(wd.columns[:2])\
            +list(wd.columns[wd.columns.get_loc(start_date):wc.columns.get_loc(end_date)+1]),\
            'uc':list(uc.columns[:6])\
            +list(uc.columns[uc.columns.get_loc(start_date):uc.columns.get_loc(end_date)+1]),\
            'ud':list(ud.columns[:6])\
            +list(ud.columns[ud.columns.get_loc(start_date):ud.columns.get_loc(end_date)+1])
}

In [182]:
wc = wc[col_list['wc']].merge(geo,on=['Country/Region','Province/State'])

In [183]:
wc[((wc['Lat']==0) & (wc['Long']==0) & (wc['Province/State'] != 'Unknown')) \
           | (wc['iso3'].isin(['USA'])) | (wc['iso3'].isnull())]

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,4/2/22,4/3/22,4/4/22,4/5/22,4/6/22,4/7/22,iso3,Lat,Long,Population
105,NaN,Diamond Princess,0,0,0,0,0,0,0,0,...,712,712,712,712,712,712,NaN,NaN,NaN,NaN
173,NaN,MS Zaandam,0,0,0,0,0,0,0,0,...,9,9,9,9,9,9,NaN,NaN,NaN,NaN
240,NaN,Summer Olympics 2020,0,0,0,0,0,0,0,0,...,865,865,865,865,865,865,NaN,35.6491,139.7737,NaN
255,NaN,US,1,1,2,2,5,5,5,6,...,80150811,80155397,80179289,80208810,80248986,80289237,USA,40.0000,-100.0000,329466283.0
279,NaN,Winter Olympics 2022,0,0,0,0,0,0,0,0,...,535,535,535,535,535,535,NaN,39.9042,116.4074,NaN


In [184]:
# drop rows above and reorder the columns 
wc.drop(wc[((wc['Lat']==0) & (wc['Long']==0) & (wc['Province/State'] != 'Unknown')) \
           | (wc['iso3'].isin(['USA'])) | (wc['iso3'].isnull())].index,inplace=True)
wc = wc[reorder_columns(wc.columns)]

In [185]:
wd = wd[col_list['wd']].merge(geo,on=['Country/Region','Province/State'])

In [186]:
wd[((wd['Lat']==0) & (wd['Long']==0) & (wd['Province/State'] != 'Unknown')) \
           | (wd['iso3'].isin(['USA'])) | (wd['iso3'].isnull())]

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,4/2/22,4/3/22,4/4/22,4/5/22,4/6/22,4/7/22,iso3,Lat,Long,Population
105,NaN,Diamond Princess,0,0,0,0,0,0,0,0,...,13,13,13,13,13,13,NaN,NaN,NaN,NaN
173,NaN,MS Zaandam,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,NaN,NaN,NaN,NaN
240,NaN,Summer Olympics 2020,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,NaN,35.6491,139.7737,NaN
255,NaN,US,0,0,0,0,0,0,0,0,...,981612,981644,982099,982576,983817,984571,USA,40.0000,-100.0000,329466283.0
279,NaN,Winter Olympics 2022,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,NaN,39.9042,116.4074,NaN


In [187]:
# drop the rows above 
wd.drop(wd[((wd['Lat']==0) & (wd['Long']==0) & (wd['Province/State'] != 'Unknown')) \
           | (wd['iso3'].isin(['USA'])) | (wd['iso3'].isnull())].index,inplace=True)
wd = wd[reorder_columns(wd.columns)]

In [188]:
uc = uc.loc[((uc['Lat']!=0) | (uc['Long']!=0)) & ((uc['iso3'].notnull())),col_list['uc']]

# groupby the values by each state and generate 2 tables: one for sum, and the other for mean values 
us_geo = uc.groupby(['iso3','Country/Region','Province/State'])[['Lat','Long']].mean()
uc_core = uc.groupby(['iso3','Country/Region','Province/State']).sum().drop(['Lat','Long'],axis=1)

# merge the mean and sum tables 
uc = pd.merge(uc_core,us_geo,on=['iso3','Country/Region','Province/State']).reset_index()

# reorder the columns 
uc = uc[reorder_columns(uc.columns)]

In [189]:
ud = ud.loc[((ud['Lat']!=0) | (ud['Long']!=0)) & ((ud['iso3'].notnull())),col_list['ud']]

# groupby the values by each state and generate 2 tables: one for sum, and the other for mean values 
ud_core = ud.groupby(['iso3','Country/Region','Province/State']).sum().drop(['Lat','Long'],axis=1)

# merge the mean and sum tables 
ud = pd.merge(ud_core,us_geo,on=['iso3','Country/Region','Province/State']).reset_index()

# reorder the columns 
ud = ud[reorder_columns(ud.columns)]

## group both global and us tables by either Confirmed or Deaths

In [190]:
def group_by_type(data_1, data_2, type):
    
    # Confirmed and Deaths
    data = pd.concat([data_1,data_2])

    # Population manipulation (No country-level statistics for multi-states)
    base_pop = 1e5
    data['num_states'] = data.groupby(['iso3','Country/Region']).iso3.transform('size')
    data.loc[(data['Province/State'].isnull()) & (data['num_states']>1),'Population'] = 0
    data.drop('num_states',axis=1,inplace=True)

    # Wide to long form transformation
    data = data.melt(id_vars=['iso3','Province/State','Country/Region','Lat','Long','Population'])
    data.rename(columns={'variable':'Date','value':type},inplace=True)
    # data['Date'] = pd.to_datetime(data['Date'],format='%m/%d/%y')
    data[type].fillna(0,inplace=True)
    data['Province/State'].replace(np.nan,'NA',inplace=True)
    
    # Reorder columns
    data = data[[data.columns[-2]]+list(data.columns[:-2])+[data.columns[-1]]]

    # per 100K (r_: Rate)
    data['r_'+type] = data[type]/data['Population']*base_pop

    # Daily changes (i_: Daily Raw, ri_: Daily Rate)
    data['i_'+type] = data[type]-data.groupby(['iso3','Province/State','Country/Region'])[type].shift(1)
    data['i_'+type].fillna(data[type],inplace=True)
    data['ri_'+type] = data['i_'+type]/data['Population']*base_pop

    # Country-level grouping
    data['Tot_'+type] = data.groupby(['Date','iso3','Country/Region'])[type].transform('sum')
    data['iTot_'+type] = data.groupby(['Date','iso3','Country/Region'])['i_'+type].transform('sum')
    data['rTot_'+type] = data['Tot_'+type]/data.groupby(['Date','iso3','Country/Region'])['Population'].transform('sum')*base_pop
    data['riTot_'+type] = data['iTot_'+type]/data.groupby(['Date','iso3','Country/Region'])['Population'].transform('sum')*base_pop

    # data.drop(data[data['Date']==data['Date'][0]].index,inplace=True)
    data = data.round(4)
    data.reset_index(drop=True,inplace=True)

    return data

In [193]:
# group tables by Confirmed or Deaths 
confirmed = group_by_type(wc,uc,'Confirmed')
deaths = group_by_type(wd,ud,'Deaths')

# merge two tables above 
covid = confirmed.merge(deaths,on=['Date','iso3','Province/State','Country/Region','Lat','Long','Population'])

In [194]:
covid.head()

,Date,iso3,Province/State,Country/Region,Lat,Long,Population,Confirmed,r_Confirmed,i_Confirmed,...,rTot_Confirmed,riTot_Confirmed,Deaths,r_Deaths,i_Deaths,ri_Deaths,Tot_Deaths,iTot_Deaths,rTot_Deaths,riTot_Deaths
0,1/22/20,AFG,NA,Afghanistan,33.9391,67.7100,38928341.0,0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0
1,1/22/20,ALB,NA,Albania,41.1533,20.1683,2877800.0,0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0
2,1/22/20,DZA,NA,Algeria,28.0339,1.6596,43851043.0,0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0
3,1/22/20,AND,NA,Andorra,42.5063,1.5218,77265.0,0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0
4,1/22/20,AGO,NA,Angola,-11.2027,17.8739,32866268.0,0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0


# save result as csv file 

In [196]:
filename = 'csse_covid_19_time_series/'+target_path['summary']
covid.to_csv(filename,index=False)

In [198]:
df = pd.read_csv('csse_covid_19_time_series/time_series_covid19.csv')

In [199]:
df.head()

,Date,iso3,Province/State,Country/Region,Lat,Long,Population,Confirmed,r_Confirmed,i_Confirmed,...,rTot_Confirmed,riTot_Confirmed,Deaths,r_Deaths,i_Deaths,ri_Deaths,Tot_Deaths,iTot_Deaths,rTot_Deaths,riTot_Deaths
0,1/22/20,AFG,NaN,Afghanistan,33.9391,67.7100,38928341.0,0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0
1,1/22/20,ALB,NaN,Albania,41.1533,20.1683,2877800.0,0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0
2,1/22/20,DZA,NaN,Algeria,28.0339,1.6596,43851043.0,0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0
3,1/22/20,AND,NaN,Andorra,42.5063,1.5218,77265.0,0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0
4,1/22/20,AGO,NaN,Angola,-11.2027,17.8739,32866268.0,0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0


In [224]:
df[df['iso3'] == 'USA']\
[{'Province/State','Deaths', 'Tot_Deaths', 'iTot_Deaths', 'rTot_Deaths'}].sort_values(by = 'Tot_Deaths', ascending = False)

,Tot_Deaths,rTot_Deaths,iTot_Deaths,Province/State,Deaths
269536,971569,295.5452,496.0,Wyoming,1796
269511,971569,295.5452,496.0,Missouri,20170
269509,971569,295.5452,496.0,Minnesota,12434
269508,971569,295.5452,496.0,Michigan,35554
269507,971569,295.5452,496.0,Massachusetts,20135
...,...,...,...,...,...
8655,1,0.0003,0.0,Minnesota,0
8656,1,0.0003,0.0,Mississippi,0
8657,1,0.0003,0.0,Missouri,0
8658,1,0.0003,0.0,Montana,0
